<a href="https://www.kaggle.com/kamaljp/tweets-nlp-modeling-inprogress?scriptVersionId=89513997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### <a id="cont"> Game Plan

Classifying a bit of sentence can become tedious when done many times. 

Computer can recognise the "tedious" word in above sentence, yet have no emotional or even computational response. We are going to try and succeed in creating such a response from the Computer, using the NLP library Spacy and ML libraries in this notebook.

[Is the dataset balanced?](#vis_1)
    
[Which country or locality has had many tweets?](#vis_2)
    
[How the sentences are represented in spaCy under the hood?](#vis_dis)
    
[Which keyword has been used in tweets to communicate the disasters?](#vis_3)
    
[Which keyword have communicated correctly when a disaster has occured?](#vis_4)

What Next?
    
    The Roots that are used in the tweets that communicate is identfied. Some of these roots create false positives and some create false negatives. Further analysis and understanding is required. Based on that, predictions will be conducted.
    
[Understanding the results](#results)


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import plotly.express as px

from spacy import displacy
from spacy.matcher import Matcher

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [4]:
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')
test.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


In [5]:
#Loading the spacy library with the small corpus model
nlp = spacy.load('en_core_web_sm')

### <a id="vis_1"> Is the dataset balanced?

In [6]:
#Lets warm up the dataset with some visuals

#Is the training dataset balanced? 

balance = train.groupby('target')['id'].count().reset_index()
balance.head()

balance.target = balance.target.apply(lambda x: str(x))

fig = px.bar(data_frame=balance, x='target',y='id',color='target')
fig.show()

[back to top](#cont)

### <a id="vis_2"> Which country or locality has had many tweets?

In [7]:
lokale = train.groupby('location')['id'].count().reset_index()
lokale.sort_values('id',ascending=False,inplace=True)

fig = px.bar(data_frame=lokale[:50], y='location',x='id',color='location')
fig.update_layout(yaxis={'categoryorder':'total descending'})
fig.show()

[back to top](#cont)

In [8]:
#Let us sample some tweets
for x in train.text[:10]:
    print(x)

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Forest fire near La Ronge Sask. Canada
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
13,000 people receive #wildfires evacuation orders in California 
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
I'm on top of the hill and I can see a fire in the woods...
There's an emergency evacuation happening now in the building across the street
I'm afraid that the tornado is coming to our area...


In [9]:
#Replacing the %20 with space
train.loc[~train.keyword.isna(),'keyword'] = train.loc[~train.keyword.isna(),'keyword'].apply(lambda x: x.replace('%20',' '))

[back to top](#cont)

### <a id="vis_dis"> How the sentences are represented in spaCy under the hood?

In [10]:
text = nlp(train.text[136])
displacy.render(text.sents, style="dep")

[back to top](#cont)

In [11]:
# The keywords can be more informative, so let us use the power of Spacy objects and lemmatize
key = nlp(train.keyword[136])

#Keyword of interest has to be a Root
for token in key:
    print(token.lemma_,token.pos_,token.dep_)

airplane NOUN compound
accident NOUN ROOT


In [12]:
keys =(_ for _ in train.loc[~train.keyword.isna(),'keyword'])

In [13]:
#Checking how the conditions work. Even though there is more than one token, it returns only the root 
for i in range(20):
    doc = nlp(next(keys))
    print(doc)
    for token in doc:
        if token.dep_ == 'ROOT':
            print(token.lemma_)

ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze
ablaze


In [14]:
#helper function to return the root keyword as a lemma. That will greatly reduce the different keywords
def get_root(key):
    doc = nlp(key)
    for token in doc:
        if token.dep_ == 'ROOT':
            return token.lemma_

In [15]:
#creating ROOT column 
train.loc[~train.keyword.isna(),'roots'] = train.loc[~train.keyword.isna(),'keyword'].apply(lambda x: get_root(x))

[back to top](#cont)

### <a id="vis_3"> Which keyword has been used in tweets to communicate the disasters?

In [16]:
key_root = train.groupby('roots')['id'].count().reset_index()
key_root.sort_values('id',ascending=False,inplace=True)

fig = px.bar(data_frame=key_root[:50], y='roots',x='id',color='roots')
fig.update_layout(yaxis={'categoryorder':'total descending'})
fig.show()

[back to top](#cont)

### <a id="vis_4"> Which keyword have communicated correctly when a disaster has occured?

In [17]:
target_root = train.groupby(['roots','target'])['id'].count().reset_index()
target_root.sort_values('id',ascending=False,inplace=True)
target_root.target = target_root.target.apply(lambda x: str(x))

In [18]:
fig = px.bar(data_frame=target_root[50:100], y='roots',x='id',color='target')
fig.update_layout(yaxis={'categoryorder':'total descending'},height=1000)
fig.show()

[back to top](#cont)

### Keywords alone are insufficient

Occurance of a keyword in a tweet by itself cannot make the tweet linked with the disaster. Take the example of "Fire". We will see what the spaCy has to offer next... 

In [19]:
# Tweet that is taking about the disaster  
tweet_1 = train.loc[(train.roots == 'fire') & (train.target == 1),'text'].values[1]
tweet_1

'@POTUS Would you please explain what you are going to do about the volcanoes &amp; bush fires spouting all that CO2 into the air?'

In [61]:
# Tweet that is taking about the disaster  
tweet_3 = train.loc[(train.roots == 'ablaze') & (train.target == 1),'text'].values[2]
tweet_3

'INEC Office in Abia Set Ablaze - http://t.co/3ImaomknnA'

In [20]:
# Tweet that is taking about the debate happening between politcians  
tweet_2 = train.loc[(train.roots == 'fire') & (train.target == 0),'text'].values[1]
tweet_2

'Ted Cruz fires back at Jeb &amp; Bush: \x89ÛÏWe lose because of Republicans like Jeb &amp; Mitt.\x89Û\x9d [Video] -  http://t.co/bFtiaPF35F'

In [21]:
#There are multiple pipes in the nlp object that evaluates the sentences. Lets take the support of NER
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7fe6b801a050>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x7fe6b801a0c0>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7fe6b8031550>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7fe6b7f9e780>),
 ('lemmatizer',
  <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x7fe6b7f996e0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fe6b8031c50>)]

In [22]:
#Lets check what entities the nlp object returns for the tweets.
doc1 = nlp(tweet_1)

for ent in doc1.ents:
    print(ent.text,ent.label_)
print('_______')
for ent in doc1:
    print(ent.text,ent.pos_,ent.dep_)
    
#Seems the bush plants are considered as the person by the small spaCy Corpus. 

bush PERSON
_______
@POTUS PROPN npadvmod
Would AUX aux
you PRON nsubj
please INTJ intj
explain VERB ccomp
what PRON dobj
you PRON nsubj
are AUX aux
going VERB ccomp
to PART aux
do VERB xcomp
about ADP prep
the DET det
volcanoes NOUN pobj
& CCONJ cc
amp PROPN conj
; PUNCT punct
bush PROPN compound
fires VERB ROOT
spouting VERB xcomp
all PRON dobj
that PRON nsubj
CO2 PROPN relcl
into ADP prep
the DET det
air NOUN pobj
? PUNCT punct


In [23]:
#Lets check what entities the nlp object returns for the tweets.
doc2 = nlp(tweet_2)

for ent in doc2.ents:
    print(ent.text,ent.label_)
print('_______')
for ent in doc2:
    print(ent.text,ent.pos_,ent.dep_)

#We can see the 2nd tweet has got more entities. 

Ted Cruz PERSON
Jeb & ORG
Bush PERSON
Republicans NORP
Jeb & ORG
_______
Ted PROPN compound
Cruz PROPN nsubj
fires VERB ROOT
back ADV advmod
at ADP prep
Jeb PROPN pobj
& CCONJ cc
amp PROPN conj
; PUNCT punct
Bush PROPN conj
: PUNCT punct
ÛÏWe ADJ amod
lose NOUN appos
because SCONJ prep
of ADP pcomp
Republicans PROPN pobj
like ADP prep
Jeb PROPN pobj
& CCONJ cc
amp PROPN conj
; PUNCT punct
Mitt.Û PROPN compound
[ X punct
Video X nmod
] X dep
- PUNCT punct
  SPACE dep
http://t.co/bFtiaPF35F NUM appos


### Seems the regular NER doesn't recognize disasters

The entities listed on the tweets miss the keywords of interest completely. The Parts of speec and the Dependency pipes give some direction.

The word fire is "NOUN" in case of disasters and other case it is a "VERB". In both cases dependency wise it is a "ROOT". That may not help.

### just using a matcher, not NER

Matchers can be created by using the keyword and then use the process Algo guy at explosion has shared.... Let me first try this...

### There are items 

There are tweets for which there is no clear keyword that indicates whether it is a disaster or not. Can we reliably extract keywords from these tweets?

In [38]:
#the pattern required is as below...let us try adding one matcher pattern
pattern = [{'LOWER': 'evacuation', 'POS': {'NOT_IN': ['VERB']}}]
type(pattern)

list

In [45]:
#create patterns using the function, by giving the keywords
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab, validate=True)

matcher.add("evac_ptn", [pattern])

In [52]:
#Adding all the roots to the matcher

for keyword in train.roots.unique()[1:]:
    pattern = [{'LOWER': keyword, 'POS': {'NOT_IN': ['VERB']}}]
    pattern_name = keyword+'pattern'
    matcher.add(pattern_name,[pattern])

In [77]:
fire = (_ for _ in train['text'] if 'fire' in _.lower())
#below loop has to be fired in seperate cell for iteration
for i in range(20):
    print(next(fire))

In [79]:
titles = (_ for _ in train['text'])

Firepower in the lab [electronic resource] : automation in the fight against infectious diseases and bioterrorism /Û_ http://t.co/KvpbybglSR
Pendleton media office said only fire on base right now is the Horno blaze.
Property losses from California wildfire nearly double as week-old blaze rages http://t.co/E0UUsnpsq5
#breaking Firefighters battling blaze at east Cary condo building http://t.co/mIM8hH2ce6
Property losses from #California wildfire nearly double as week-old blaze rages: The fireÛ_ http://t.co/MsdizftZ2g
Bright &amp; BLAZING Fireman Birthday Party http://t.co/9rFo9GY3nE #Weddings
Max blew tf up ! ?????? shots fired ???? #CatfishMTV
Did anyone else see that fireball falling to earth? Look like a plane blew up.
BREAKING: Fairfax County firefighter placed on admin leave amid probe into Facebook post about putting police in 'body bags' dept. says.
UPDATE: Va. firefighter on administrative leave after Facebook post calls for people to put officers in 'body bags.' http://t.co/

In [68]:
for i in range(20):
    doc = nlp(next(titles))
    if len(matcher(doc)) == 1:
        print(doc)

Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/0TSlQjOKvh via @theadvocatemag #LGBT
@sayn_ae angel or arson
Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel http://t.co/Q4L7Dg56JM
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/2Y9dnP5vtg via @theadvocatemag #LGBT | https://t.co/6XuL6DCOsh
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/UBFr1URAFc #LGBT | https://t.co/AlnV51d95x
Mourning notices for stabbing arson victims stir Û÷politics of griefÛª in Israel http://t.co/GbluHRrlto
Owner of Chicago-Area Gay Bar Admits to Arson Scheme http://t.co/ZPxE3fMYNG #LGBT
Arson suspect linked to 30 fires caught in Northern California http://t.co/wnuqQAtTTP (via @latimes)
Arson suspect linked to 30 fires caught in Northern California - Los Angeles Times http://t.co/PrRB4fhXtv
Arson suspect linked to 30 fires caught in Northern California http://t.co/u1fuWrGK5U
#NOWPLAYING Arsonist MC -  So Impressed -  @ARSONIS

In [42]:
doc = nlp(trial.text.values[3])
for match_id, start, end in matcher(doc):
    print(doc[start: end])

evacuation


In [80]:
from IPython.display import HTML as html_print

def style(s, bold=False):
    blob = f"<text>{s}</text>"
    if bold:
        blob = f"<b style='background-color: #fff59d'>{blob}</b>"
    return blob

def html_generator(g, n=10):
    blob = ""
    for i in range(n):
        doc = next(g)

        state = [[t, False] for t in doc]
        for idx, start, end in matcher(doc):
            for i in range(start, end):
                state[i][1] = True
        blob += style(' '.join([style(str(t[0]), bold=t[1]) for t in state]) + '<br>') 
    return blob

In [82]:
g = (d for d in nlp.pipe(titles) if matcher(d))
html_print(html_generator(g, n=10))

In [89]:
train['Pred']= train.text.apply(lambda d: 1 if len(matcher(nlp(d))) > 0 else 0)

In [95]:
train.sample(10)

,id,keyword,location,text,target,roots,Pred
1419,2047,casualties,everywhere,Revise the Death to America scenario? \n\nWhil...,1,casualty,1
456,659,attack,Dubai,@etribune US Drone attack kills 4-suspected m...,1,attack,1
4954,7060,meltdown,NaN,Looks like it may have been microsofts anti vi...,0,meltdown,1
953,1380,body bag,New York,New Ladies Shoulder Tote Handbag Women Cross B...,0,bag,1
6242,8914,snowstorm,London,Can you list 5 reasons why a London #TubeStrik...,0,snowstorm,1
2069,2970,dead,Glasgow,@soapscoop i need you to confirm that ross is ...,0,dead,1
4327,6146,hijack,NaN,REVEALED: Everton hijack United bid for 14-yea...,0,hijack,1
1638,2366,collapsed,they/her,+ DID YOU SAY TO HIM!!?!?!?!' and phil actuall...,0,collapse,0
6121,8737,sinking,London,Slowly sinking wasting ?? @edsheeran,0,sink,0
5007,7141,military,highlands&slands scotland,Bad News for US: China Russia Bolstering Milit...,0,military,1


In [93]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(train['target'], train['Pred'])

array([[2008, 2334],
       [ 821, 2450]])

In [94]:
print(classification_report(train['target'], train['Pred']))

              precision    recall  f1-score   support

           0       0.71      0.46      0.56      4342
           1       0.51      0.75      0.61      3271

    accuracy                           0.59      7613
   macro avg       0.61      0.61      0.58      7613
weighted avg       0.62      0.59      0.58      7613



Where the matchers fail due to in-sufficient capability to see the context surrounding itself...

In [97]:
mistakes = (train.loc[lambda d: d['Pred'] == 1].loc[lambda d: d['target'] == 0]['text'])

html_print(html_generator((nlp(i) for i in mistakes), n=21))

In [98]:
def parse_train_data(doc):
    detections = [(doc[start:end].start_char, doc[start:end].end_char, 'disaster') for idx, start, end in matcher(doc)]
    return (doc.text, {'entities': detections})

parse_train_data(nlp("I reject the laws of the misguided false prophets imprison nations fueling self annihilation"))

('I reject the laws of the misguided false prophets imprison nations fueling self annihilation',
 {'entities': [(80, 92, 'disaster')]})

In [99]:
titles = train.loc[lambda d: d['target'] == 1]['text']

In [100]:
TRAIN_DATA = [parse_train_data(d) for d in nlp.pipe(titles) if len(matcher(d)) == 1]
TRAIN_DATA[5:8]

[("I'm afraid that the tornado is coming to our area...",
  {'entities': [(20, 27, 'disaster')]}),
 ('Three people died from the heat wave so far',
  {'entities': [(32, 36, 'disaster')]}),
 ('#Flood in Bago Myanmar #We arrived Bago',
  {'entities': [(1, 6, 'disaster')]})]

In [105]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("en")
    ner = nlp.add_pipe('ner')
    #nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [107]:
import random 
import datetime as dt
from spacy.training import Example

nlp = create_blank_nlp(TRAIN_DATA)
optimizer = nlp.begin_training()  
for i in range(20):
    random.shuffle(TRAIN_DATA)
    losses = {}

    for text, annotations in TRAIN_DATA:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example],sgd=optimizer, losses=losses)
    print(f"Losses at iteration {i} - {dt.datetime.now()}", losses)

[2022-03-07 17:03:34,348] [INFO] Created vocabulary
[2022-03-07 17:03:34,350] [INFO] Finished initializing nlp object


Losses at iteration 0 - 2022-03-07 17:04:17.100922 {'ner': 861.3638633155387}
Losses at iteration 1 - 2022-03-07 17:04:59.927589 {'ner': 188.63013989661005}
Losses at iteration 2 - 2022-03-07 17:05:42.660240 {'ner': 131.66502047391944}
Losses at iteration 3 - 2022-03-07 17:06:25.026587 {'ner': 103.40991975643412}
Losses at iteration 4 - 2022-03-07 17:07:09.552182 {'ner': 56.52443143554058}
Losses at iteration 5 - 2022-03-07 17:07:52.580180 {'ner': 57.53847143672265}
Losses at iteration 6 - 2022-03-07 17:08:35.675021 {'ner': 46.48526435953866}
Losses at iteration 7 - 2022-03-07 17:09:19.976954 {'ner': 26.81664975024509}
Losses at iteration 8 - 2022-03-07 17:10:04.820803 {'ner': 36.12966553141808}
Losses at iteration 9 - 2022-03-07 17:10:49.844561 {'ner': 17.119364959954122}
Losses at iteration 10 - 2022-03-07 17:11:35.048594 {'ner': 15.46713665395352}
Losses at iteration 11 - 2022-03-07 17:12:24.269949 {'ner': 16.763683699683796}
Losses at iteration 12 - 2022-03-07 17:13:11.637378 {'ner

In [108]:
nlp.pipeline

[('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7fe6b115ae50>)]

In [109]:
doc = nlp("I'm afraid that the tornado is coming to our area...")
displacy.render(doc, style="ent")

In [ ]:
from spacy.training import Example

example = Example.from_dict(nlp.make_doc(text), annotations)
nlp.update([example])

In [24]:
def has_dis_token(doc):
    doc = nlp(doc)
    for t in doc:
        if t.lower_ in train.keyword:
            print(t)
            if t.pos_ == 'NOUN':
                return True
    #if the sentence has got no token that is noun and matching the keyword, then it is a general tweet
    return False

In [25]:
#Lets test with couple of tweets, say 1st 10 training data
trial = train.iloc[:10,:]
trial['result'] = trial.text.apply(lambda x: has_dis_token(x))
trial

,id,keyword,location,text,target,roots,result
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,NaN,False
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,NaN,False
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,NaN,False
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,NaN,False
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,NaN,False
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,NaN,False
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,NaN,False
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,NaN,False
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,NaN,False
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,NaN,False


In [26]:
trial.text.values

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       '13,000 people receive #wildfires evacuation orders in California ',
       'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
       '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
       '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
       "I'm on top of the hill and I can see a fire in the woods...",
       "There's an emergency evacuation happening now in the building across the street",
       "I'm afraid that the tornado is coming to our area..."],
      dtype=object)

In [33]:
def get_parts(doc):
    tec=[]
    for t in doc:
        tec.append((t.text,t.pos_,t.dep_))
        
    return tec

In [34]:
#Let us take these tweets and try to see if there is any pattern that we can use
[get_parts(d) for d in nlp.pipe(trial.text)]

[[('Our', 'PRON', 'poss'),
  ('Deeds', 'NOUN', 'nsubj'),
  ('are', 'AUX', 'ROOT'),
  ('the', 'DET', 'det'),
  ('Reason', 'PROPN', 'attr'),
  ('of', 'ADP', 'prep'),
  ('this', 'DET', 'det'),
  ('#', 'NOUN', 'compound'),
  ('earthquake', 'NOUN', 'pobj'),
  ('May', 'AUX', 'aux'),
  ('ALLAH', 'PROPN', 'nsubj'),
  ('Forgive', 'VERB', 'advcl'),
  ('us', 'PRON', 'dobj'),
  ('all', 'PRON', 'appos')],
 [('Forest', 'NOUN', 'compound'),
  ('fire', 'NOUN', 'ROOT'),
  ('near', 'ADP', 'prep'),
  ('La', 'PROPN', 'compound'),
  ('Ronge', 'PROPN', 'compound'),
  ('Sask', 'PROPN', 'pobj'),
  ('.', 'PUNCT', 'punct'),
  ('Canada', 'PROPN', 'ROOT')],
 [('All', 'DET', 'det'),
  ('residents', 'NOUN', 'nsubj'),
  ('asked', 'VERB', 'ROOT'),
  ('to', 'ADP', 'prep'),
  ("'", 'PUNCT', 'punct'),
  ('shelter', 'NOUN', 'pobj'),
  ('in', 'ADP', 'prep'),
  ('place', 'NOUN', 'pobj'),
  ("'", 'PUNCT', 'punct'),
  ('are', 'AUX', 'aux'),
  ('being', 'AUX', 'auxpass'),
  ('notified', 'VERB', 'xcomp'),
  ('by', 'ADP', 'agen